# 数据库

常见的数据库分为两种

+ 非关系型数据库
+ 关系数据库

我们就分两块来研究

## 关系数据库与orm

关系数据库一般都是用orm来操作,毕竟这样更加直观,最常见的orm是sqlalchemy,但它太重了,这边就不讲了,我会介绍两个相对轻的orm

### pony.orm

[Pony](https://docs.ponyorm.com/firststeps.html)是一个独特的ORM,它提供了干净的接口和select表达式的列表解析式检索功能,非常的pythonic,更有卖点的是它还额外提供了一个根据类图直接生成代码的工具,非常适合原型开发.

In [2]:
from pony import orm

pony只支持绝对地址引入数据库

In [1]:
import os

In [3]:
db = orm.Database()

In [4]:
class User(db.Entity):
    name = orm.Required(unicode)
    password = orm.Required(unicode)
    type = orm.Required(unicode,default=u"local")

In [5]:
orm.show(User)

class User(Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(unicode)
    password = Required(unicode)
    type = Required(unicode, default=u'local')


与数据库文件绑定:

In [6]:
db.bind('sqlite', os.path.abspath("codes/c6/pony_db.sqlite"),create_db=True)

建立映射:

In [7]:
db.generate_mapping(create_tables=True)

#### 创建表格

> 新增数据

In [8]:
p1 = User(name='John',password="1234")
p2 = User(name='Mary', password="2234")
p3 = User(name='Bob',password="3234")

In [9]:
orm.commit()

#### 查询

In [10]:
orm.select(p for p in User)[:]

[User[1], User[2], User[3]]

In [11]:
orm.select(p for p in User).show()

id|name|password|type 
--+----+--------+-----
1 |John|1234    |local
2 |Mary|2234    |local
3 |Bob |3234    |local


In [12]:
User.select().show()

id|name|password|type 
--+----+--------+-----
1 |John|1234    |local
2 |Mary|2234    |local
3 |Bob |3234    |local


In [13]:
User.get(name="John")

User[1]

#### 更新数据

In [14]:
User.get(name="John").password="4234"

In [15]:
orm.commit()

In [16]:
User.get(name="John").password

u'4234'

#### 删除数据

In [17]:
User.get(id=3).delete()
orm.commit()

In [18]:
User.select().show()

id|name|password|type 
--+----+--------+-----
1 |John|4234    |local
2 |Mary|2234    |local


#### 定义关系

> 一对一

一对一关系有两种:

+ Optional-Required:

+ Optional-Optional:

```python
class Person(db.Entity):
    passport = orm.Optional("Passport")
class Passport(db.Entity):
    person = orm.Required("Person")
```

> 一对多

一对多也是两种可能

+ Set-Required

+ Set-Optional


   
```python   
class Order(db.Entity):
    items = Set("OrderItem")

class OrderItem(db.Entity):
    order = Required(Order)
```

> 多对多

多对多就是用Set

```python
class Product(db.Entity):
    tags = Set("Tag")

class Tag(db.Entity):
    products = Set(Product)
        
```

> 提及自己

这种类型分为堆成关系和非对称关系,比如你是你妈妈的孩子,你妈妈不是你的孩子,这种就是非对称关系,比如你是你老婆的配偶 ,你老婆也是你的配偶,这就是对称关系,这两种又可以分成一对多,一对一这两种,也就是一共四种关系模型

```python
class Person(db.Entity):
    name = Required(str)
    spouse = Optional("Person", reverse="spouse") # 配偶在Person中找,是一对一(要不就犯法了),对称关系
    friends = Set("Person", reverse="friends")    # 好友也在Person中找,是一对多,对称关系
    manager = Optional("Person", reverse="employees") # 主管,一对一,非对称
    employees = Set("Person", reverse="manager") # 员工,一对多,非对称

```

> 多重关系

多重关系比较复杂了,像下面这个推特的模型,用户写的推特和用户之间的关系就要像如下的定义方式

```python
class User(db.Entity):
    tweets = Set("Tweet", reverse="author")
    favorites = Set("Tweet", reverse="favorited")

class Tweet(db.Entity):
    author = Required(User, reverse="tweets")
    favorited = Set(User, reverse="favorites")
```

注意:

在非交互环境下,所有增删改查操作必须在`db_session`中进行,可以使用`with db_session`或者将其作为修饰符

#### ER图原型设计

pony官方还给了一个原型设计的工具https://editor.ponyorm.com/

##### 用pony改造uesr的增删改查操作

In [19]:
%%writefile codes/c6/app.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify
from flask_restful import Api,Resource
import json

app = Flask(__name__)
api = Api(app)

import os

from pony import orm

db = orm.Database()
class User(db.Entity):
    name = orm.Required(unicode)
    password = orm.Required(unicode)
    type = orm.Required(unicode,default=u"local")
    
    def __str__(self):
        return "<USER: id-{self.id}--name-{self.name}>".format(self=self)
    def __repr__(self):
        return self.__str__()
    
db.bind('sqlite', os.path.abspath("pony_db.sqlite"),create_db=True)
db.generate_mapping(create_tables=True)


@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404

class UsersAPI(Resource):
    
    def get(self):
        with orm.db_session :
            result = map(lambda x:x.__str__(),User.select()[:])
        return {"result":result}
     
    @orm.db_session
    def post(self):
        data = json.loads(request.data)
        user = User(name = data.get("name"),password =data.get("password"))
        return jsonify({"result":"save {name} done!".format(name=user.name)})
    
class UserAPI(Resource):
    @orm.db_session
    def get(self,user_id):
        
        user = User.get(id=user_id)
        if not user:
            abort(404)
            
        result = {"name":user.name,"password":user.password,"type":user.type}
        return jsonify({"result":result})
    
    @orm.db_session   
    def put(self,user_id):
        data = request.args
        user =User.get(id=user_id)
        if request.args.get("name"):
            user.name=data.get("name")
        if request.args.get("password"):
            user.password=data.get("password")
        
        return jsonify({"result":"update {name} done!".format(name=user_id)})  
    @orm.db_session
    def delete(self,user_id):
        
        User.get(id=user_id).delete()
        return jsonify({"result":"delete {id} done!".format(id=user_id)})  
    
    
api.add_resource(UsersAPI, '/users/')        
api.add_resource(UserAPI, '/user/<int:user_id>')  

if __name__ == '__main__':
    app.run(debug=True)




Overwriting codes/c6/app.py


In [1]:
import requests
import json

In [2]:
requests.post("http://127.0.0.1:5000/users/",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'result': u'save niuniu done!'}

In [3]:
requests.get("http://127.0.0.1:5000/users/").json()

{u'result': [u'<USER: id-1--name-John>',
  u'<USER: id-2--name-Mary>',
  u'<USER: id-4--name-niuniu>']}

In [4]:
requests.get("http://127.0.0.1:5000/user/2").json()

{u'result': {u'name': u'Mary', u'password': u'2234', u'type': u'local'}}

### peewee

[peewee](http://docs.peewee-orm.com/en/latest/)是python的超轻量级ORM,它对数据库io效率的影响比较小,而且关键的接口干净,非常适合和写RESTFUL服务

安装:

pip install peewee

### 数据库连接

peewee支持主流的mysql,sqlite3,和postgresql基本功能,并且可以通过插件支持postgresql的全新json特性,也支持shema迁移等特性.

以sqlite为例:

In [5]:
from peewee import SqliteDatabase
import os

In [6]:
db = SqliteDatabase(os.path.abspath("codes/c6/peewee_db.sqlite"))

# 基本功能:

创建映射类:

映射类必须继承自Model或者它的子类,各个字段的定义也是直观的`XxxField()`的样式,定义的字段类型有:

Field Type	|Sqlite|	Postgresql|	MySQL
---|---|---|---
CharField	|varchar	|varchar|	varchar
FixedCharField	|char	|char|	char
TextField	|text|	text	|longtext
DateTimeField|	datetime	|timestamp|	datetime
IntegerField	|integer|	integer	|integer
BooleanField	|integer|	boolean|	bool
FloatField|	real	|real	|real
DoubleField	|real	|double precision|	double precision
BigIntegerField	|integer|	bigint|	bigint
SmallIntegerField	|integer	|smallint	|smallint
DecimalField	|decimal|	numeric|	numeric
PrimaryKeyField	|integer|	serial|	integer
ForeignKeyField	|integer|	integer|	integer
DateField	|date|	date	|date
TimeField	|time	|time	|time
BlobField	|blob	|bytea	|blob
UUIDField	|text	|uuid	|varchar(40)
BareField	|untyped	|not supported|	not supported

里面可以选择是否加入参数说明

通用的参数有:

参数|取值|默认值|说明
---|---|---
null | 布尔值|False|是否可以为空
index |布尔值|False|是否创建索引
unique |布尔值| False|是否唯一 
verbose_name |str|None|别名 
help_text | str|None |帮助文档
db_column|str| None | 数据库中的名字
default|object|None|默认值 
choices|tuple|None|可选值
primary_key|布尔值|False|主键 
sequence|---|None|序列,但必须要数据库本身支持
constraints|---|None|限制,例子:[Check('price > 0')]
schema |---| None |可选的schema名字,但必须要数据库本身支持

特殊参数:

Field type	|Special Parameters
---|---
CharField|	max_length
FixedCharField	|max_length
DateTimeField	|formats
DateField	|formats
TimeField	|formats
DecimalField	|max_digits, decimal_places, auto_round, rounding
ForeignKeyField	|rel_model, related_name, to_field, on_delete, on_update, extra
BareField	|coerce



In [7]:
from peewee import Model,CharField,DateField,BooleanField,ForeignKeyField

In [8]:
class User(Model):
    name = CharField()
    password = CharField()
    type = CharField(default="local")

    class Meta:
        database = db # 确定使用的是哪个db,方便多种数据库配合使用

#### 创建会话

In [9]:
db.connect()

#### 创建对应的数据库

In [10]:
db.create_tables([User])

#### 增删改查

##### 增 

In [11]:
p1 = User(name='John',password="1234")
p2 = User(name='Mary', password="2234")
p3 = User(name='Bob',password="3234")

In [12]:
p1.save()
p2.save()
p3.save()

1

 使用save()方法将会将这条数据保存到数据库,并返回它的主键,另一种方法是使用映射类的静态方法(`create()`)

In [13]:
User.create(name='Tom',password="4234")

##### 查

In [14]:
john = User.get(User.name=='John')

In [15]:
john.name

u'John'

In [16]:
for user in User.select():
    print user.name

John
Mary
Bob
Tom


也可以用where方法来做筛选

In [17]:
[p.name for p in User.select().where(User.name == 'Bob')]

[u'Bob']

##### 改

In [18]:
john = User.get(User.name=='John')
john.password = "5234"
john.save()

1

直接修改属性就可以了

##### 删

In [19]:
john.delete_instance()

1

In [20]:
[user.name for user in list(User.select())]

[u'Mary', u'Bob', u'Tom']

##### 用peewee改造uesr的增删改查操作

In [1]:
%%writefile codes/c6/app2.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify
from flask_restful import Api,Resource
import json

app = Flask(__name__)
api = Api(app)

import os

from peewee import SqliteDatabase
db = SqliteDatabase(os.path.abspath("peewee_db.sqlite"))
from peewee import Model,CharField,DateField,BooleanField,ForeignKeyField
class User(Model):
    name = CharField()
    password = CharField()
    type = CharField(default="local")
    def __str__(self):
        return "<USER: id-{self.id}--name-{self.name}>".format(self=self)
    def __repr__(self):
        return self.__str__()
    class Meta:
        database = db # 确定使用的是哪个db,方便多种数据库配合使用
        
db.connect()    

@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404

class UsersAPI(Resource):
    
    def get(self):
        result = [user.__str__() for user in list(User.select())]
        return {"result":result}

    def post(self):
        data = json.loads(request.data)
        user = User(name = data.get("name"),password =data.get("password")).save()
        return jsonify({"result":"save {name} done!".format(name=data.get("name"))})
    
class UserAPI(Resource):

    def get(self,user_id):
        
        user = User.get(User.id==user_id)
        if not user:
            abort(404)
            
        result = {"name":user.name,"password":user.password,"type":user.type}
        return jsonify({"result":result})
    
  
    def put(self,user_id):
        data = request.args
        user = User.get(User.id==user_id)
        if request.args.get("name"):
            user.name=data.get("name")
        if request.args.get("password"):
            user.password=data.get("password")
        user.save()
        
        return jsonify({"result":"update {name} done!".format(name=user_id)})  

    def delete(self,user_id):
        
        User.get(User.id==user_id).delete_instance() 
        return jsonify({"result":"delete {id} done!".format(id=user_id)})  
    
    
api.add_resource(UsersAPI, '/users/')        
api.add_resource(UserAPI, '/user/<int:user_id>')  

if __name__ == '__main__':
    app.run(debug=True)


Overwriting codes/c6/app2.py


In [29]:
import requests
import json

In [30]:
requests.post("http://127.0.0.1:5000/users/",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'result': u'save niuniu done!'}

In [2]:
requests.get("http://127.0.0.1:5000/users/").json()

NameError: name 'requests' is not defined

In [32]:
requests.get("http://127.0.0.1:5000/user/2").json()

{u'result': {u'name': u'Mary', u'password': u'2234', u'type': u'local'}}

## 非关系数据库

非关系数据库我们只讲最常用的redis和mongodb,他们适合独立使用

### Redis

[Redis](http://www.redis.net.cn/tutorial/3503.html)一般用于将数据保存在缓存中,是速度最快的非关系数据库之一,
常用的库是redis

查看redis是否开启可以使用`ps -ef | grep redis`命令,

redis是键值对非关系数据库,详细的使用方法可以参考

In [35]:
import redis

In [36]:
User = redis.StrictRedis(host='localhost', port=6379, db=0)

In [37]:
User.hmset(1,{"name":'John',"password":"1234","type":"local"})
User.hmset(2,{"name":'Mary', "password":"2234","type":"local"})
User.hmset(3,{"name":'Bob', "password":"3234","type":"local"})

True

In [38]:
User.hgetall(1)

{'name': 'John', 'password': '1234', 'type': 'local'}

In [39]:
User.hgetall(2)

{'name': 'Mary', 'password': '2234', 'type': 'local'}

In [44]:
User.keys()

['3', '2', '1']

In [47]:
User.hvals(1)

['1234', 'local', 'John']

In [16]:
User.hmset(1,{"name":'John',"password":"12434","type":"local"})

True

In [17]:
User.hgetall(1)

{'name': 'John', 'password': '12434', 'type': 'local'}

#### 使用Redis作为数据库改造user增删改查服务

In [54]:
%%writefile codes/c6/app3.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify
from flask_restful import Api,Resource
import json

app = Flask(__name__)
api = Api(app)

import redis
User = redis.StrictRedis(host='localhost', port=6379, db=0)
        
@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404

class UsersAPI(Resource):
    
    def get(self):
        result = zip(User.keys(),map(lambda x:User.hgetall(x),User.keys()))
        return {"result":result}

    def post(self):
        _id = max(map(lambda x:int(x),User.keys()))+1
        data = json.loads(request.data)
        data["type"]="local"
        User.hmset(_id,data)
        return jsonify({"result":"save {name} done!".format(name=data.get("name"))})
    
class UserAPI(Resource):

    def get(self,user_id):
        
        user = User.hgetall(user_id)
        if not user:
            abort(404)
            

        return jsonify({"result":user})
    
  
    def put(self,user_id):
        data = request.args
        user = User.hgetall(user_id)
        if not request.args.get("name"):
            data["name"]=user.get("name")
            
        if not request.args.get("password"):
            data["password"]=user.get("password")
            
        User.hmset(user_id,data)
        
        return jsonify({"result":"update {name} done!".format(name=user_id)})  

    def delete(self,user_id):
        
        User.delete(user_id)
        return jsonify({"result":"delete {id} done!".format(id=user_id)})  
    
    
api.add_resource(UsersAPI, '/users/')        
api.add_resource(UserAPI, '/user/<int:user_id>')  

if __name__ == '__main__':
    app.run(debug=True)

Overwriting codes/c6/app3.py


In [50]:
import requests
import json

In [51]:
requests.post("http://127.0.0.1:5000/users/",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'result': u'save niuniu done!'}

In [55]:
requests.get("http://127.0.0.1:5000/users/").json()

{u'result': [[u'3',
   {u'name': u'Bob', u'password': u'3234', u'type': u'local'}],
  [u'2', {u'name': u'Mary', u'password': u'2234', u'type': u'local'}],
  [u'4', {u'name': u'niuniu', u'password': u'qwe', u'type': u'local'}],
  [u'1', {u'name': u'John', u'password': u'1234', u'type': u'local'}]]}

In [53]:
requests.get("http://127.0.0.1:5000/user/2").json()

{u'result': {u'name': u'Mary', u'password': u'2234', u'type': u'local'}}

### mongodb

mongodb大名鼎鼎了,可以直接操作它的是[pymongo](https://api.mongodb.com/python/current/tutorial.html#querying-for-more-than-one-document)

基本使用：

首先是用`ps -ef | grep mongod`查看mongodb是否在运行,如果不在运行,先开服务

#### 创建连接

In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

不加参数则是默认连接到本地默认端口

#### 连接到数据库

In [3]:
db = client.User

#### 连接到数据库的表(collection)

In [4]:
User = db.User

#### 增加文档(一行数据)

In [65]:
User.insert_many([{"name":'John',"password":"1234","type":"local"},
                 {"name":'Mary', "password":"2234","type":"local"},
                 {"name":'Bob', "password":"3234","type":"local"}])

#### 查看数据

In [5]:
[i for i in User.find()]

[{u'_id': ObjectId('574c2150b7e0c715d0b57bce'),
  u'name': u'John',
  u'password': u'1234',
  u'type': u'local'},
 {u'_id': ObjectId('574c2150b7e0c715d0b57bcf'),
  u'name': u'Mary',
  u'password': u'2234',
  u'type': u'local'}]

#### 修改数据

In [81]:
bob=User.find_one({"name":"Bob"})

In [82]:
bob["password"]= "12365"

In [83]:
bob

{u'_id': ObjectId('574c2150b7e0c715d0b57bd0'),
 u'name': u'Bob',
 u'password': '12365',
 u'type': u'local'}

In [92]:
User.find_one_and_replace({"name":'Bob'},{"name":'Bob', "password":"123457","type":"local"})

{u'_id': ObjectId('574c2150b7e0c715d0b57bd0'),
 u'name': u'Bob',
 u'password': u'3234',
 u'type': u'local'}

In [93]:
[i for i in User.find()]

[{u'_id': ObjectId('574c2150b7e0c715d0b57bce'),
  u'name': u'John',
  u'password': u'1234',
  u'type': u'local'},
 {u'_id': ObjectId('574c2150b7e0c715d0b57bcf'),
  u'name': u'Mary',
  u'password': u'2234',
  u'type': u'local'},
 {u'_id': ObjectId('574c2150b7e0c715d0b57bd0'),
  u'name': u'Bob',
  u'password': u'123457',
  u'type': u'local'}]

#### 删除数据

In [106]:
from bson.objectid import ObjectId 

In [107]:
User.find_one_and_delete({'_id':ObjectId('574c2150b7e0c715d0b57bd0')})

{u'_id': ObjectId('574c2150b7e0c715d0b57bd0'),
 u'name': u'Bob',
 u'password': u'123457',
 u'type': u'local'}

In [108]:
[i for i in User.find()]

[{u'_id': ObjectId('574c2150b7e0c715d0b57bce'),
  u'name': u'John',
  u'password': u'1234',
  u'type': u'local'},
 {u'_id': ObjectId('574c2150b7e0c715d0b57bcf'),
  u'name': u'Mary',
  u'password': u'2234',
  u'type': u'local'}]

#### 存储文件(GridFS)

In [6]:
import gridfs

In [7]:
db2 = client.Imgs

In [8]:
fs = gridfs.GridFS(db2)

In [54]:
with open("1.jpg","rb") as f:
    data = f.read()
a = fs.put(data,format="jpg",filename="1.jpg",owner="hsz",content_type="image/jpeg")

In [15]:
b =fs.get(a).read()

In [22]:
fs.exists(filename="1.jpg")

True

In [51]:
f2 = fs.find_one({"filename":"1.jpg"})

In [45]:
f3 = fs.find_one({"filename":"12.jpg"})

In [46]:
True if f3 else False

False

In [52]:
f2.content_type

In [32]:
f2.format

u'jpg'

In [30]:
data= f2.read()
with open("2.jpg","wb") as f:
    f.write(b)

In [34]:
fs.list()

[u'1.jpg', u'11.jpg']

In [39]:
fs.find({"owner":"hsz"})[0].name

u'1.jpg'

> 写一个图片存储的rest服务器

In [137]:
%%writefile codes/c6/appImg.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify,make_response
from flask_restful import Api,Resource
import json

app = Flask(__name__)
api = Api(app)

from pymongo import MongoClient
import gridfs
from bson.objectid import ObjectId 

client = MongoClient('mongodb://localhost:27017/')
imgdb = client.Imgs
fs = gridfs.GridFS(imgdb)
@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404

content_types={
    "jpg":"image/jpeg",
    "png":"image/png",
    "bmp":"application/x-bmp",
    "gif":"image/gif"
}

class ImgsAPI(Resource):
    
    def get(self):
        names = fs.list()
        result = []
        
        objs = [fs.find_one({"filename":i}) for i in names]
        for j in objs:
            imginfo = {}
            imginfo["filename"]=j.name
            imginfo["id"]=str(j._id)
            result.append(imginfo)
        return {"result":result}

    def post(self):
        file_ = request.files['file']
        filename = file_.filename
        format_= filename.decode("utf-8").split(".")[-1].encode("utf-8")
        fs.put(file_.read(),format=format_,filename=filename,owner="hsz",content_type=content_types.get(format_))
        return {"result":"save {name} done!".format(name=filename)}
    
class ImgAPI(Resource):

    def get(self,filename):
        
        img = fs.find_one({"filename":filename})
        if not img:
            abort(404)
            
        imgdata = fs.get(img._id).read()
        response = make_response(imgdata)
        response.headers["Content-Disposition"] = "attachment; filename={filename}".format(filename=filename)

        return response
        

    def delete(self,filename):
        
        fs.delete(fs.find_one({"filename":filename})._id)
        return {"result":"delete {id} done!".format(id=filename)}
    
    
api.add_resource(ImgsAPI, '/api/imgs/')        
api.add_resource(ImgAPI, '/api/img/<string:filename>')  

if __name__ == '__main__':
    app.run(host="0.0.0.0",debug=True)

Overwriting codes/c6/appImg.py


In [60]:
import requests
import json

In [107]:
requests.get("http://127.0.0.1:5000/api/imgs/").json()

{u'result': [{u'filename': u'1.jpg', u'id': u'574c2c40b7e0c717a6f6ae8f'},
  {u'filename': u'11.jpg', u'id': u'574d1835b7e0c71df0bf68f3'},
  {u'filename': u'111.jpg', u'id': u'574d34d1b7e0c72479811cb9'},
  {u'filename': u'2.jpg', u'id': u'574d3002b7e0c7240819d40a'}]}

In [105]:
files = {'file': ("111.jpg",open("1.jpg","rb"))}

In [106]:
requests.post("http://127.0.0.1:5000/api/imgs/",files=files)

<Response [200]>

In [127]:
requests.get("http://127.0.0.1:5000/api/imgs/").json()

{u'result': [{u'filename': u'1.jpg', u'id': u'574c2c40b7e0c717a6f6ae8f'},
  {u'filename': u'11.jpg', u'id': u'574d1835b7e0c71df0bf68f3'},
  {u'filename': u'111.jpg', u'id': u'574d34d1b7e0c72479811cb9'},
  {u'filename': u'2.jpg', u'id': u'574d3002b7e0c7240819d40a'}]}

In [135]:
requests.delete("http://127.0.0.1:5000/api/img/2.jpg").json()

{u'result': u'delete 2.jpg done!'}

In [136]:
requests.get("http://127.0.0.1:5000/api/imgs/").json()

{u'result': [{u'filename': u'1.jpg', u'id': u'574c2c40b7e0c717a6f6ae8f'},
  {u'filename': u'11.jpg', u'id': u'574d1835b7e0c71df0bf68f3'},
  {u'filename': u'111.jpg', u'id': u'574d34d1b7e0c72479811cb9'}]}